# Training Demo

Here we demonstrate how to train Popari downloading and preprocessing a multisample spatial transcriptomics dataset for analysis with Popari. In particular, we will be working with the **Alzheimer's Disease (AD)** dataset from the "Preprocessing Demo" notebook.

In [1]:
# Disable warnings for prettier notebook
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pathlib import Path

import torch

import popari
from popari.model import Popari
from popari import pl, tl

In [3]:
data_directory = Path("/path/to/directory/")

In [6]:
# data_directory = Path("/work/magroup/shahula/spatiotemporal_transcriptomics_integration/data/STARmapPlus/SCP1375/")

In [7]:
K = 10
dataset_path = data_directory / f"preprocessed_dataset.h5ad"
context = {"device": "cuda:1", "dtype": torch.float64}

popari_example = Popari(
    K=K,
    dataset_path=dataset_path,
    torch_context=context,
    initial_context=context,
    verbose=1
)

[2023/05/24 23:52:14]	 Initializing MetageneState
[2023/05/24 23:52:14]	 Initializing SpatialAffinityState
[2023/05/24 23:52:14]	 Initializing EmbeddingOptimizer
[2023/05/24 23:52:14]	 Initializing EmbeddingState
[2023/05/24 23:52:14]	 Initializing metagenes and hidden states
Current number of clusters: 9
Resolution: 1.0
Current number of clusters: 31
Resolution: 3.1622776601683795
Current number of clusters: 16
Resolution: 1.7782794100389228
Current number of clusters: 12
Resolution: 1.333521432163324
Current number of clusters: 10
Resolution: 1.1547819846894583


## Pretraining

In [8]:
# Initialization with NMF
for preiteration in range(10):
    print(f"----- Preiteration {preiteration} -----")
    popari_example.estimate_parameters(update_spatial_affinities=False)
    popari_example.estimate_weights(use_neighbors=False)

----- Preiteration 0 -----
[2023/05/24 23:56:03]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/24 23:56:04]	 Updating sigma_yx
[2023/05/24 23:56:04]	 Updating latent states


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

----- Preiteration 9 -----
[2023/05/24 23:56:16]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/24 23:56:16]	 Updating sigma_yx
[2023/05/24 23:56:16]	 Updating latent states


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
# Reinitialize spatial affinities
pretrained_embeddings = [popari_example.embedding_optimizer.embedding_state[dataset.name] for dataset in popari_example.datasets]
popari_example.parameter_optimizer.spatial_affinity_state.initialize(pretrained_embeddings)

## Hierarchical Training

Using hierarchical mode, we can train Popari more robustly using a lower resolution view of the original spatial transcriptomics data. We can then "superresolve" the embeddings at the higher resolution to regain a fine-grained view.

In [16]:
K = 10
dataset_path = data_directory / f"preprocessed_dataset.h5ad"
context = {"device": "cuda:1", "dtype": torch.float64}
hierarchical_levels = 2
superresolution_lr = 1e-1

hierarchical_example = Popari(
    K=K,
    dataset_path=dataset_path,
    torch_context=context,
    initial_context=context,
    hierarchical_levels=hierarchical_levels,
    superresolution_lr=superresolution_lr,
    verbose=1
)

[2023/05/25 00:04:02]	 Initializing MetageneState
[2023/05/25 00:04:02]	 Initializing SpatialAffinityState
[2023/05/25 00:04:02]	 Initializing EmbeddingOptimizer
[2023/05/25 00:04:02]	 Initializing EmbeddingState
[2023/05/25 00:04:02]	 Initializing metagenes and hidden states
Current number of clusters: 9
Resolution: 1.0
Current number of clusters: 31
Resolution: 3.1622776601683795
Current number of clusters: 16
Resolution: 1.7782794100389228
Current number of clusters: 12
Resolution: 1.333521432163324
Current number of clusters: 10
Resolution: 1.1547819846894583
[2023/05/25 00:04:37]	 Initializing hierarchy level 1
[2023/05/25 00:04:38]	 Downsized dataset from 8186 to 1298 spots.
[2023/05/25 00:04:39]	 Downsized dataset from 10372 to 1167 spots.
[2023/05/25 00:04:39]	 Initializing MetageneState
[2023/05/25 00:04:39]	 Initializing SpatialAffinityState
[2023/05/25 00:04:39]	 Initializing EmbeddingOptimizer
[2023/05/25 00:04:39]	 Initializing EmbeddingState
[2023/05/25 00:04:39]	 Initial

In [17]:
# Initialization with NMF
for preiteration in range(10):
    print(f"----- Preiteration {preiteration} -----")
    hierarchical_example.estimate_parameters(update_spatial_affinities=False)
    hierarchical_example.estimate_weights(use_neighbors=False)

----- Preiteration 0 -----
[2023/05/25 00:04:42]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/25 00:04:42]	 Updating sigma_yx
[2023/05/25 00:04:42]	 Updating latent states


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

----- Preiteration 9 -----
[2023/05/25 00:04:55]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/25 00:04:55]	 Updating sigma_yx
[2023/05/25 00:04:56]	 Updating latent states


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
# Reinitialize spatial affinities
pretrained_embeddings = [hierarchical_example.embedding_optimizer.embedding_state[dataset.name] for dataset in hierarchical_example.datasets]
hierarchical_example.parameter_optimizer.spatial_affinity_state.initialize(pretrained_embeddings)

In [19]:
for iteration in range(50):
    print(f"----- Iteration {iteration} -----")
    hierarchical_example.estimate_parameters()
    hierarchical_example.estimate_weights()

----- Iteration 0 -----
[2023/05/25 00:04:56]	 Updating spatial affinities


Updating Σx-1:   0%|          | 0/1000 [00:00<?, ?it/s]

[2023/05/25 00:04:59]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/25 00:04:59]	 Updating sigma_yx
[2023/05/25 00:05:00]	 Updating latent states


...

[2023/05/25 00:10:33]	 Updating metagenes


Updating M:   0%|          | 0/10000 [00:00<?, ?it/s]

[2023/05/25 00:10:33]	 Updating sigma_yx
[2023/05/25 00:10:34]	 Updating latent states


Updating weight w/ neighbors:   0%|          | 0/1000 [00:00<?, ?it/s]

Updating weight w/ neighbors:   0%|          | 0/1000 [00:00<?, ?it/s]

In [21]:
hierarchical_example.superresolve(n_epochs=5000, tol=1e-6)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

Updating Σx-1:   0%|          | 0/1000 [00:00<?, ?it/s]

### Save results to disk

In [39]:
hierarchical_example.save_results(data_directory / f"hierarchical_resuts.h5ad")

### Load a pretrained model

In [40]:
from popari.model import load_trained_model

In [41]:
reloaded_model = load_trained_model(data_directory / f"hierarchical_resuts.h5ad")

[2023/05/25 07:02:57]	 Initializing MetageneState
[2023/05/25 07:02:57]	 Initializing SpatialAffinityState
[2023/05/25 07:02:57]	 Initializing EmbeddingOptimizer
[2023/05/25 07:02:57]	 Initializing EmbeddingState
[2023/05/25 07:02:59]	 Initializing MetageneState
[2023/05/25 07:02:59]	 Initializing SpatialAffinityState
[2023/05/25 07:02:59]	 Initializing EmbeddingOptimizer
[2023/05/25 07:02:59]	 Initializing EmbeddingState
[2023/05/25 07:03:00]	 Initializing MetageneState
[2023/05/25 07:03:00]	 Initializing SpatialAffinityState
[2023/05/25 07:03:00]	 Initializing EmbeddingOptimizer
[2023/05/25 07:03:00]	 Initializing EmbeddingState
